In [ ]:
import faiss
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import faiss
from langchain.agents.middleware import dynamic_prompt, ModelRequest
import xml.etree.ElementTree as ET
import pyttsx3
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from typing import List
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
import re
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import numpy as np
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
import spacy
import pickle
import requests
from langchain_core.documents import Document
from bs4 import BeautifulSoup
import re
from scispacy.linking import EntityLinker
import scispacy
import re
from langchain.messages import SystemMessage,HumanMessage,AIMessage,ToolMessage
from langchain.tools import tool
from langchain_community.docstore.in_memory import InMemoryDocstore
import pyobo
import time
from collections import defaultdict
from typing import List, Optional, Literal
import re
from typing import Optional


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/bionik/miniconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#linker = pyobo.get_scispacy_entity_linker("uniprot", filter_for_definitions=False, resolve_abbreviations=True)
nlp = spacy.load('en_ner_jnlpba_md')
nlp.disable_pipes("tagger", "parser")
nlp.add_pipe("sentencizer")
linker = pyobo.get_scispacy_entity_linker("hgnc", filter_for_definitions=False, resolve_abbreviations=True,)


In [3]:
def pubmed_query(
    include_topics: List[str],
    operator: Literal["AND", "OR"] = "AND",
    exclude_topics: Optional[List[str]] = None,
    publication_type: Optional[Literal["review", "article"]] = None,
    organism: Optional[str] = None,
    journal: Optional[str] = None,
    start_year: Optional[int] = None,
    end_year: Optional[int] = None,
    free_full_text: bool = True
) -> str:
    """
    Build a simple, elegant PubMed query.
    """

    clauses = []

    # Automatic free full text
    if free_full_text:
        clauses.append("free full text[filter]")

    # Include topics
    if include_topics:
        topic_clause = " OR ".join([f'"{t}"[Title/Abstract]' for t in include_topics])
        if len(include_topics) > 1:
            topic_clause = f'({topic_clause})'
        clauses.append(topic_clause)

    # Publication type
    if publication_type:
        pt_map = {"review": "review[Publication Type]", "article": "journal article[Publication Type]"}
        clauses.append(pt_map.get(publication_type.lower(), ""))

    # Organism
    if organism:
        org_map = {"human": "humans[MeSH Terms]", "mouse": "mice[MeSH Terms]", "rat": "rats[MeSH Terms]"}
        clauses.append(org_map.get(organism.lower(), f'"{organism}"[MeSH Terms]'))

    # Journal
    if journal:
        clauses.append(f'"{journal}"[Journal]')

    # Date range
    if start_year and end_year:
        clauses.append(f'("{start_year}"[Date - Publication] : "{end_year}"[Date - Publication])')
    elif start_year:
        clauses.append(f'"{start_year}"[Date - Publication]')
    elif end_year:
        clauses.append(f'"{end_year}"[Date - Publication]')

    # Exclude topics
    if exclude_topics:
        for t in exclude_topics:
            clauses.append(f'NOT "{t}"[Title/Abstract]')

    # Combine clauses with AND
    return " AND ".join(clauses)


In [4]:
query = pubmed_query(
    include_topics=["AKT signaling",'wnt signalling'],
    publication_type="review",
    organism="Human",
    start_year=2020,
    end_year=2025
)

In [5]:
def fetch_ncbi_data(query):
    base_url_esearch='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    params_esearch = {"db": "pubmed","term": query,"retmode": "json","retstart": 0,"retmax": 10000,'email':'your_email@example.com',
              'datetype':'pdat','mindate':'2010/01/01', 'maxdate':'2025/01/01'}
    response_esearch=requests.get(base_url_esearch, params=params_esearch)
    list_of_pubmed=response_esearch.json()['esearchresult']['idlist']
    dois=[]
    journal=[]
    base_url_efetch='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    for list in [list_of_pubmed[i:i+10]for i in range(0,len(list_of_pubmed),10)]:
        query=f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        param_efetch={'retmode':'xml',"db": "pubmed","id": ','.join(list),"retstart": 0,"retmax": 1000}
        response_efetch=requests.get(base_url_efetch,params=param_efetch)
        tree = ET.fromstring(response_efetch.text)
        base='https://doi.org/'
        for i in tree.findall('PubmedArticle/PubmedData/ArticleIdList/ArticleId'):
            if i.attrib.get('IdType')=='doi':
                dois.append(base+i.text)
        for i in tree.findall('PubmedArticle/MedlineCitation/Article/Journal/Title'):
            journal.append(i.text)
            
    return dois, journal
            

In [ ]:
doi,jornal=fetch_ncbi_data(query)

232

In [1]:
def load_doi_page_raw(url: str):
    loader = WebBaseLoader(
        web_paths=[url],
        header_template={
            "User-Agent": (
                "Mozilla/5.0 (X11; Linux x86_64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/120.0 Safari/537.36"
            )
        },
    )
    docs = loader.load()
    return docs[0]

def extract_main_article_text(html_text: str) -> str:
    soup = BeautifulSoup(html_text, "lxml")

    # Remove junk
    for tag in soup([
        "script", "style", "noscript", "svg",
        "header", "footer", "nav", "aside",
        "form", "button"
    ]):
        tag.decompose()

    text = soup.get_text(separator="\n")

    # Normalize whitespace
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = re.sub(r"[ \t]+", " ", text)

    return text.strip()

def build_clean_document(raw_doc: Document) -> Document:
    clean_text = extract_main_article_text(raw_doc.page_content)

    return Document(
        page_content=clean_text,
        metadata={
            **raw_doc.metadata,
            "source_type": "doi_webpage",
            "cleaned": True,
        }
    )
def load_full_article_from_doi(url: str) -> Document:
    raw_doc = load_doi_page_raw(url)
    clean_doc = build_clean_document(raw_doc)
    return clean_doc


NameError: name 'Document' is not defined

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
}
all_docs=[]
for i in tqdm(doi): 
    try:
        doc=WebBaseLoader(i,continue_on_failure=True,requests_kwargs={'allow_redirects': True,
                                            "headers": headers}).load()[0]
        if len(doc.page_content)>2000:
            all_docs.append(doc)
    except Exception as e:
        print(e)
        continue
    
    
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
}
all_docs=[]
for i in tqdm(doi): 
    try:
        doc=load_full_article_from_doi(i)
        if len(doc.page_content)>1000:
            all_docs.append(doc)
    except Exception as e:
        print(e)
        continue



 87%|████████▋ | 201/232 [07:16<01:11,  2.31s/it]

('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|██████████| 232/232 [08:20<00:00,  2.16s/it]


In [ ]:
def extract_main_content(text: str) -> Optional[str]:
    
    abstract_pat = r'\babstract\b'
    intro_pat = r'\bintroduction\b'
    ref_pat = r'\breference*\b'
    ack_pat = r'\backnowledgment*\b'

    start_matches = list(re.finditer(intro_pat, text, re.IGNORECASE))
    if not start_matches:
        start_matches = list(re.finditer(abstract_pat, text, re.IGNORECASE))

    end_matches = list(re.finditer(ack_pat, text, re.IGNORECASE))
    if not end_matches:
        end_matches = list(re.finditer(ref_pat, text, re.IGNORECASE))

    if not start_matches or not end_matches:
        return None

    candidates = []

    for start in start_matches:
        for end in end_matches:
            if end.start() > start.start():
                span = text[start.start():end.start()].strip()
                if len(span) > 300:
                    candidates.append(span)

    if not candidates:
        return None

    return max(candidates, key=len)


In [57]:
cleaned_docs=[]
for doc in all_docs:
    text=extract_main_content(doc.page_content)
    if text and len(text)>2000:
        cleaned_nlp=linker(nlp(text))
        new_text=text
        for i in reversed(cleaned_nlp.ents):
            id=i._.kb_ents
            if id:
                name=linker.kb.cui_to_entity[id[0][0]].canonical_name
                new_text = new_text[:i.start_char] + name + ' ' + i.label_ + new_text[i.end_char:]
        doc.page_content=new_text
        cleaned_docs.append(doc)

/home/bionik/miniconda3/envs/LLM/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=700,chunk_overlap=250)
chunks=splitter.split_documents(cleaned_docs)

In [ ]:
embeddings_model = OllamaEmbeddings(model="qwen3-embedding:0.6b",validate_model_on_init=True)
embedding_dim = len(embeddings_model.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
ids = vector_store.add_documents(documents=chunks)

KeyboardInterrupt: 

In [ ]:
vector_store.save_local("/home/bionik/AI_ML/FDA_Project/LLM/faiss_index")

In [ ]:
vector_store = FAISS.load_local(
    "/home/bionik/AI_ML/FDA_Project/LLM/faiss_index",
    embeddings_model,
    allow_dangerous_deserialization=True 
)

In [ ]:
LLM_model = ChatOllama(num_ctx=10000,model="qwen3:4b-instruct-2507-q4_K_M",validate_model_on_init=True)


In [ ]:
@tool
def rewrite_query_tool(query: str, history: str) -> str:
    """
    Rewrite biomedical query for better literature retrieval.
    Agent can choose whether to use it.
    """
    prompt = f"""
Rewrite the following biomedical question for literature retrieval.
- Preserve meaning exactly.
- Use conversation history if helpful.
- If already optimal, return unchanged.
- Return ONLY the rewritten query.

Conversation history:
{history}

Original question:
{query}
"""
    return LLM_model.invoke(prompt).content.strip()

# -------------------------------
# 3️⃣ Retrieval Middleware
# -------------------------------
@dynamic_prompt
def retrieve_context_middleware(request: ModelRequest) -> str:
    """
    Always retrieves context for all queries from a pre-built vector store.
    """
    user_question = request.state["messages"][-1].content.strip()
    history = "\n".join(
        m.content for m in request.state["messages"][:-1]
        if m.type in {"human", "ai"}
    )

    # -------------------------------
    # vector_store should be pre-built externally
    # -------------------------------
    # Example usage:
    # docs = vector_store.as_retriever(search_type='mmr', search_kwargs={"k":10,"fetch_k":20}).get_relevant_documents(user_question)
    docs = vector_store.max_marginal_relevance_search(user_question)

    # Format sources with citation tags
    sources = "\n\n".join(f"[SOURCE {i+1} reaserch paper: {d.metadata['source']}]\n{d.page_content}" for i, d in enumerate(docs))

    # Save in state for dynamic system prompt
    request.state["retrieved_context"] = sources
    return ""  # Middleware doesn't return prompt directly

# -------------------------------
# 4️⃣ Dynamic System Prompt
# -------------------------------
@dynamic_prompt
def rag_system_prompt(request: ModelRequest) -> str:
    messages = request.state["messages"]
    user_question = messages[-1].content.strip()
    context = request.state.get("retrieved_context", "NO_RELEVANT_SOURCES_FOUND")
    history = "\n".join(
        m.content for m in messages[:-1] if m.type in {"human", "ai"}
    )

    # -----------------------
    # Query classification
    # -----------------------
    classify_prompt = f"""
Classify the following biomedical question.

Return ONLY one word:
FACTUAL  -> asks for known mechanisms or established facts
REASONING -> asks about consequences, interventions, or what-if scenarios

Question:
"{user_question}"
"""
    qtype = LLM_model.invoke(classify_prompt).content.strip().upper()

    # -----------------------
    # Mode-specific instructions
    # -----------------------
    if qtype == "FACTUAL":
        mode_instruction = """
FACTUAL MODE (STRICT BUT COMPLETE):

- Use retrieved SOURCES as the PRIMARY evidence.
- If SOURCES do not fully explain the mechanism, you MAY use
  well-established general biomedical knowledge to complete the explanation.
- Any such content MUST be explicitly labeled as:

  "General biomedical knowledge (not from retrieved sources):"

- General knowledge MUST:
  - be widely accepted textbook-level biology
  - not introduce speculation or uncertainty
  - not contradict SOURCES

- Cite all statements supported by SOURCES as [SOURCE X].
"""
    else:  # REASONING
        mode_instruction = """
REASONING MODE:

- Use SOURCES to ground all factual statements and cite them as [SOURCE X].
- You MAY use general biomedical knowledge when needed; label it clearly.
- You MAY propose mechanistic reasoning or hypotheses.
- All speculative content MUST be explicitly labeled as:

  "Hypothesis:"
"""

    # -----------------------
    # Final system prompt
    # -----------------------
    return f"""
You are a biomedical research assistant.

Global rules:
- Prefer retrieved SOURCES over general knowledge.
- Never invent citations.
- If evidence is insufficient, say so explicitly.
- Provide links to all SOURCES at the end of the answer.

Question type: {qtype}

User question:
{user_question}

Conversation history:
{history}

Retrieved SOURCES:
{context}

{mode_instruction}

The agent may use the tool "rewrite_query_tool" if it improves retrieval.

Produce a clear, structured, citation-grounded answer.
"""



In [ ]:
checkpointer = InMemorySaver()

agent = create_agent(
    model=LLM_model,
    tools=[rewrite_query_tool],
    checkpointer=checkpointer,
    middleware=[
        retrieve_context_middleware,  # always inject retrieved context
        rag_system_prompt,            # dynamic prompt
        SummarizationMiddleware(
            model=LLM_model,
            trigger=("tokens", 5000),
            keep=("messages", 10),
        ),
    ],
)




In [ ]:
config = {"configurable": {"thread_id": "biomed_rag"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "How is EGFR signaling activated?"}]},
    config,
)

print(response["messages"][-1].content)

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What happens after that on molecular level??"}]},
    config,
)

print(response["messages"][-1].content)

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what if we RAS protein is mutated in when EGFR pathway is in inactive state?"}]},
    config,
)

print(response["messages"][-1].content)